In [1]:
import os 
import yaml
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.agent_toolkits import JsonToolkit, create_json_agent
from langchain_community.tools.json.tool import JsonSpec
import difflib

In [2]:
model = AzureChatOpenAI(azure_endpoint="https://firstsenseai.openai.azure.com",
                      azure_deployment="gpt-4o",
                     api_version= "2024-02-15-preview",
                     api_key= "a823ef8205364125a194795f870da84c",
                     temperature=0,
                     max_tokens=None)

In [3]:
prefix = """You are an agent designed to interact with JSON. Your goal is to return a final  
answer by interacting with the JSON.\nYou have access to the following tools which help you learn \
more about the JSON you are interacting with.\nOnly use the below tools. Only use the information \
returned by the below tools to construct your final answer.\nDo not make up any information that is \
not contained in the JSON.\nYour input to the tools should be in the \
form of `dict_['instructions'][0]` where `instructions` is the list of Python dictionary with the keys like 'scenario','steps','verify','actions'  you are interacting with, \
and the syntax used is Python. Tables should be referred in ['instructions'][0]['table']['rows'] \nYou should only use keys that you know for a fact exist.\
You must validate that a key exists by seeing it previously when calling \
`json_spec_list_keys`. \nIf you have not seen a key in one of those responses,\
you cannot use it.\nYou navigate all the key and verify the all the condition for the given input question 
and propose valid action as per the condition passed.If \
the question does not seem to be related to the JSON, just return "I don\'t know" \
as the answer.\nAlways begin your interaction with the `json_spec_list_keys` tool with \
input "data" to see what keys exist in the JSON.\n\nNote that sometimes the value at a \
given path is large. In this case, you will get an error "Value is a large dictionary, \
should explore its keys directly".\nIn this case, you should ALWAYS follow up by \
using the `json_spec_list_keys` tool to see what keys exist at that path.\nDo not simply \
refer the user to the JSON or a section of the JSON, as this is not a valid answer. \
Keep digging until you find the answer and explicitly return it.\n"""


suffix ="""Begin!"\n\nQuestion: {input}\nThought: I should look at the keys that exist \
in data to see what I have access to\n{agent_scratchpad}"""

In [4]:
@tool
def create_agent(file_path: str, query: str) -> str: 
    """
    Create an agent to extract relevant information from a specified document.
    This function initializes an agent with the document located at the given
    file path and processes the provided query to extract information related
    to it. The agent will analyze the content of the document to identify
    relevant responses based on the query.

    Args:
        file_path (str): The path to the document file that the agent will analyze.
        query (str): The question or query to be processed by the agent.

    Returns:
        Any: The response generated by the agent based on the provided query. 
             This may include relevant information or references from the document.
    """
    print("#####CREATE AGENT TOOL#####")

    with open(file_path) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
    json_spec = JsonSpec(dict_=data, max_value_length=4000)
    json_toolkit = JsonToolkit(spec=json_spec)

    json_agent_executor = create_json_agent(prefix=prefix, suffix=suffix,
        llm=model, toolkit=json_toolkit, verbose=True
    )

    response = json_agent_executor.run(query)

    return response
    

@tool
def get_file_name(sentence: str) -> str:
    """
    Retrieve the file path of a referenced document based on a given sentence.

    This function is used when there is a reference to a different document 
    within a sentence (e.g., "Refer to the ..."). It processes the sentence
    to extract and return the correct file path of the referenced document.

    Args:
        sentence (str): A sentence that may contain a reference to a document.

    Returns:
        str: The file path of the document being referenced.
    """
    print("#####GET FILE NAME TOOL#####")

    directory_path = 'json-files'
    yaml_files = [file for root, dirs, files in os.walk(directory_path)
              for file in files if file.endswith(('.json'))]

    normalized_text = sentence.lower()
    
    return f'json-files/{difflib.get_close_matches(normalized_text, yaml_files, n=1, cutoff=0.0)[0]}'


@tool 
def decision_maker(text: str) -> str:
    """
    Args:
        sentence (str): A sentence that may contain a reference to a document.

    Use this function before returning the final_response to the user

    Returns:
        str: Decision string either to use 'get_file_name' tool or return the final response as it..
    """
    print("###DECISION MAKER TOOL###")
    prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that helps in selecting correct tools.
     Following are the available tools:
     - get_file_name
     - create_agent

     You will be given a text:
     - if it contains instruction like refer to some document then you have to return "Use `get_file_name` for the <actual_sentence> and then use `create_agent` tool.
     - if it does not contain any reference then you have simply return the sentence as it is.
     """),
    ("user", "Decide for: {text}")
])
    
    return prompt_template.invoke({"text": text})


In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an agent designed to extract relevant information from documents based on queries. Your goal is to analyze the documents and return the most pertinent information related to the query. 

You have access to the following tools to assist you in this task:
- `create_agent`: This tool takes a file path and a query, creates an agent with the specified file, and runs it to extract information relevant to the query.
- `get_file_name`: This tool takes a sentence and returns the file path of the document referenced in that sentence.
- `decision_maker`: This tool takes the final response and decides whether if it should be the Final Answer or if needs to pass to any other tool.

When using the `create_agent` tool, if you encounter a response that contains the word "refer" or any indication that a document is referenced, you must immediately use the `get_file_name` tool to retrieve the correct file path of the document being referenced. 

Always start by processing the query with the `create_agent` tool to extract relevant information. If any references seem unclear or incorrect, follow up with the `get_file_name` tool to provide the correct file path.

If the query does not relate to the available documents, respond with "I don’t know" as your answer.

Only return the Final Answer if it does not contain any "refer" word that is referring to another document. Make sure to validate any references before finalizing your response.
         
Always use `decision_maker` tool before returning the Final Answer.
         """),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [create_agent, get_file_name, decision_maker]
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(handle_parsing_errors=True,agent=agent, tools=tools)

In [6]:
query ="""file_path: json-files/Authorization Business Process - All States and LOBs - SOP.json, query: Action for Claim 1108"""
response = agent_executor.invoke({"input": query})

#####CREATE AGENT TOOL#####


C:\Users\gurunaml\AppData\Local\Temp\ipykernel_7932\2223327305.py:29: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = json_agent_executor.run(query)




> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: "data"
Observation: ['SOP_No', 'Title', 'Effective_Date', 'Department', 'Product(s)', 'State(s)', 'Final_Approver_Name', 'Final_Approver_Title', 'Date', 'Technical_Writer', 'Document_Owner(s)', 'Document_Approvers', 'Internal_Quality_Reviewer', 'IQR_Date', 'Overview', 'Table of Contents', 'Definitions', 'NIA', 'PCS', 'Utilization Management (UM) (aka Utilization Review)', 'UM', 'References', 'Notes', 'AuthorizationRequirements', 'EffectiveDateOfService', 'Location', 'Authorization', 'Facilities', 'Authorization Exclusions for Florida - HH and PH Ambulance Transportation', 'Authorization Exclusion for Kentucky: Par and Non-Par Emergent/Urgent Services', 'Authorization Exclusions for Texas', 'Authorization Business Process', 'Billed level of care is higher than authorized level of care', 'Billed level of care is lower than authorized level of care', 'Billed type of stay is not in same category as authorized

In [7]:
response

{'input': 'file_path: json-files/Authorization Business Process - All States and LOBs - SOP.json, query: Action for Claim 1108',
 'output': 'The action for Claim 1108 is to pay $84.63.'}